# Data Generator (Saved in CSV)
   
Add values of $\alpha$, $\beta$, Length, Grid points, and Amplitude in the function.
The Wave Number is None for now

In [72]:
import pandas as pd
import numpy as np
from math import erf as erf_, sin, exp as exp_
from numpy import pi, sqrt, random
import numba as nb
from scipy.io import savemat
import warnings
import torch
warnings.filterwarnings('ignore')


def DataGenerator(alpha, beta=0.1, L=6, t=500, N=200, amp=0, CSV=False):
    %run ./Data_Generator/KFRequation_u.ipynb
    par = {
    'alpha': alpha,
    'beta': beta,
    'L': L,
    'N': N,
    'amp': amp
    }
    for wn in [None]:
        par['wn'] = wn
        tmax = 3000 if wn == 0.01 else t
        kfr_equation(tmax=tmax, **par)
        
    def loader(alpha, amp):
        _path = '../comp_res'
        fullpath = os.path.join(_path)
        filename = f'amp_{amp}' + f'alpha_{alpha}' 
        shock=np.load(os.path.join(filename + '.npz'))
        return shock
    
    shock = loader(par['alpha'], 0)
    @nb.vectorize(('f8(f8)'))
    def erf(x):
        return erf_(x)

    L=L
    N=N
    dx = L / N  # grid size

    x = np.empty(N + 7)
    x[0:3] = [(-L + dx * i) for i in [-3, -2, -1]]  # left ghost points
    x[3:N + 4] = [(-L + dx * i) for i in range(N + 1)]  # internal grid points on [-L,0]
    x[N + 4:N + 7] = np.arange(1, 4) * dx  # grid points ahead of the shock
    
    t=shock['t']
    u=shock['u']
    t=np.array([t])
    x=np.array([x])
    
    matDict = {"t": t.T, "x": x.T, "u": np.transpose(u)}
    
    if CSV==True:
        filename = f'amp_{amp}' + f'alpha_{alpha}'
        U_steadyState = u[:,-1] 
        pd.DataFrame(t).to_csv(filename + '_T.csv')
        pd.DataFrame(U_steadyState).to_csv(filename + '_U_steadyState.csv')
    else:
        print("Completed")
        #filename = f'amp_{amp}' + f'alpha_{alpha}'
        #savemat(filename + '.mat', matDict)
    return u, t

In [83]:
#KFR Try-3: Like RNN+FNO, Training Data is different. For one parameter, different random points in time. 
#mapping between the (input) and (Ndt times the input)


def DataGenerator_Alpha(Alpha, TrainingSamples, Ndt, Space = 5000, InitialSolve=350, dx=207): 

    X_Data = torch.zeros([TrainingSamples,dx])
    Y_Data = torch.zeros([TrainingSamples,dx])
    count = 0
    U, T_array = DataGenerator(Alpha, t=InitialSolve, CSV=False)
    #start = -Space
    for i in range(0,TrainingSamples):
    #print('i', i)
        start = int(random.uniform(-Space,-(Ndt+1)))
        X_Data[count,:] = ((torch.from_numpy(U)))[start,:].reshape(dx)
        Y_Data[count,:] = ((torch.from_numpy(U)))[start+Ndt,:].reshape(dx)
        start += 3
        count += 1
    
    Udict = {"input": X_Data, "u_results": Y_Data, "u_original": U, "time": T_array}

        
    np.save('Data_Generator/' + f'KFR_data_{Alpha}_{InitialSolve}',Udict)
    np.save('Data_Generator/' + f'input_{Alpha}_{InitialSolve}_{Ndt}_{TrainingSamples}_{Space}',X_Data)
    np.save('Data_Generator/' + f'u_results_{Alpha}_{InitialSolve}_{Ndt}_{TrainingSamples}_{Space}',Y_Data)
    np.save('Data_Generator/' + f'u_original_{Alpha}_{InitialSolve}',U)
    np.save('Data_Generator/' + f'time_{Alpha}_{InitialSolve}',T_array)
    
    return Udict

amp_0alpha_5.169 =-=-=-=-=-= 66.78s
Completed
